# Imports

In [457]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import sys
import copy
import re
import csv
from neo4j import GraphDatabase
from neo4j.exceptions import CypherSyntaxError
from SPARQLWrapper import SPARQLWrapper, JSON  # pip install sparqlwrapper

# Defined functions

In [458]:
def get_sparql_results(endpoint_url, query):
    # Helper function
    user_agent = f"WDQS-example Python/{sys.version_info[0]}.{sys.version_info[1]}"  # https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [459]:
def preprocess_input_list(input_list):
    # Helper function
    augmented = copy.deepcopy(input_list)
    for name in input_list:
        name_without_parentheses = re.sub(r"\([^()]*\)", "", name)
        if name != name_without_parentheses:
            augmented.append(name_without_parentheses)
    return augmented

In [460]:
def scrape_qcodes(list, humans_only=False, first_only=False):
    """
    Gets Q codes and urls for wikidata pages from the first page of search results.
    
    list: list of search names, eg. ['Fryderyk Chopin', 'Ferenc Liszt', 'Wojciech Żywny']
    first_only: if True, gets data from the first link (result) only
    returns: list of dicts (search_name, page_name, description, Qcode, wiki_url)
             if page with given name doesn't exist, its qcode is ommited
    """
    list = preprocess_input_list(list)
    retv = []
    input_length = len(list)
    for i, search_name in enumerate(list):
        print(f"Scraping... {i+1}/{input_length} ... " + search_name)
        url = f"https://www.wikidata.org/w/index.php?go=Go&search={search_name}"
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        for item in soup.find_all('li', class_="mw-search-result mw-search-result-ns-0"):
            page_name = item.find('a', title=True)['title'].lstrip('\u200e').split('\u200e')[0]
            try:
                description = item.find('a', title=True)['title'].lstrip('\u200e').split('\u200e')[2]
            except IndexError:
                description = None
            href = item.find('a', href=True).get('href')
            Qcode = href.lstrip('/wiki/')
            wiki_url = f"https://www.wikidata.org{href}"
            if humans_only:
                human_test = get_all_human_objects([Qcode])
                if human_test:
                    if Qcode not in [d['Qcode'] for d in retv]:
                        retv.append({'search_name': search_name, 'page_name': page_name, 'description':description, 'Qcode': Qcode, 'wiki_url': wiki_url,})
                    if first_only:
                        break
            else:
                if Qcode not in [d['Qcode'] for d in retv]:
                    retv.append({'search_name': search_name, 'page_name': page_name, 'description':description, 'Qcode': Qcode, 'wiki_url': wiki_url,})
                if first_only:
                    break
    return retv

In [461]:
def get_all_triples_and_qcodes(Qcode, allow_literals=True):
    """
    Args:
        Qcode: wikidata subject Qcode, eg.: 'Q255'
        allow_literals: whether to allow returning literal objects (objects not represented as wikidata/wikimedia pages) associated with subject's Qcode

    Returns (First value returned): 
        list of tuples representig RDF triples (object-predicate-subject) and labels associated with each wikidata entity

        Example tuple representing RDF triple:
            ('http://www.wikidata.org/entity/Q255',
            'http://www.wikidata.org/entity/P19',
            'http://www.wikidata.org/entity/Q586',
            'Ludwig van Beethoven',
            'place of birth',
            'Bonn')
        Tuple contents:
            (object,
            predicate,
            subject,
            objectLabel,
            predicateLabel,
            subjectLabel)

    Returns (Second value returned): 
        list of all Qcodes of wikidata objects associated with provided subject (Qcode)

        Example:
            ['Q255', Q'1268']
    """
    #  UWAGA:  Z jakiegoś powodu query poniżej zwraca wielokrotnie zduplikowane wyniki, pewnie błąd leży po stronie twórców biblioteki SPARQLWrapper.
    #          To samo zapytanie wprwadzowne w https://query.wikidata.org/ zwraca normalne, nie powielane wyniki.
    #          Poprawiłem to dopiero przy dodawaniu krotek do listy trójek RDF.
    

    if isinstance(Qcode, list):
        list_of_wdqcodes = ["wd:" + qcode for qcode in Qcode]
        query = """SELECT DISTINCT ?s ?sLabel ?wd ?wdLabel ?o ?oLabel   WHERE {{
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,pl". }}
        VALUES ?s {{ {} }}
        ?s ?wdt ?o.
        ?wd wikibase:directClaim ?wdt.
        ?wd rdfs:label ?wdLabel.
        FILTER((LANG(?wdLabel)) = "en")
        }}
        ORDER BY (xsd:integer(STRAFTER(STR(?wd), "http://www.wikidata.org/entity/P")))""".format(' '.join(list_of_wdqcodes))
    else:
        query = """SELECT DISTINCT ?s ?sLabel ?wd ?wdLabel ?o ?oLabel   WHERE {
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en,pl". }
        VALUES ?s { wd:%s }
        ?s ?wdt ?o.
        ?wd wikibase:directClaim ?wdt.
        ?wd rdfs:label ?wdLabel.
        FILTER((LANG(?wdLabel)) = "en")
        }
        ORDER BY (xsd:integer(STRAFTER(STR(?wd), "http://www.wikidata.org/entity/P")))"""%(Qcode)
    # print(query) 
    endpoint_url = "https://query.wikidata.org/sparql"
    results = get_sparql_results(endpoint_url, query)

    # list_of_wdqcodes = ["wd:" + qcode for qcode in list_of_qcodes]
    # query_check_objects = """SELECT ?qcode ?qcodeLabel WHERE {{
    # SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,pl". }}
    # VALUES ?qcode {{ {} }}
    # ?qcode wdt:P31 wd:Q5.
    # }}""".format(' '.join(list_of_wdqcodes))

    list_of_triples = []
    for result in results["results"]["bindings"]:
        # print(result['wdLabel']['value'])
        try:
            if result['wdLabel']['value'].__contains__('date'):  # np. Personendatenbank Germania Sacra tu wchodzi, ale to nie data...
                if "http://www.wikidata.org/.well-known" in result['oLabel']['value']:  # http://www.wikidata.org/.well-known/genid/c9a8b2f22783c60e6bf383200d9285f7 czasem są randomowe cosie w miejsach na daty
                    # t = (result['s']['value'], result['wd']['value'], result['o']['datatype'], result['sLabel']['value'], result['wdLabel']['value'], "NoData")
                    # stop = True
                    continue
                else:
                    t = (result['s']['value'], result['wd']['value'], result['o']['datatype'], result['sLabel']['value'], result['wdLabel']['value'], result['oLabel']['value'])
        except KeyError: # Personendatenbank Germania Sacra... 
            pass  # Personendatenbank Germania Sacra... 
        if not allow_literals:
            if result['o']['type'] != 'literal':
                t = (result['s']['value'], result['wd']['value'], result['o']['value'], result['sLabel']['value'], result['wdLabel']['value'], result['oLabel']['value'])
        else:
            t = (result['s']['value'], result['wd']['value'], result['o']['value'], result['sLabel']['value'], result['wdLabel']['value'], result['oLabel']['value'])
        if t not in list_of_triples:
            list_of_triples.append(t)

        # ^ WTF, ten kod wyżej nie ma sensu, nie wiem co myślałem jak to pisałem i ... czemu to działa to z datami? Jeśli result['wdLabel']['value'].__contains__('date') i allow_literals=True, czemu nie daje to błędów? 

    object_qcodes = []
    for t in list_of_triples:
        object_url = t[2]
        if "http://www.wikidata.org/entity/Q" in object_url:
            Qcode = object_url[object_url.rfind('Q'):]
            object_qcodes.append(Qcode)

    return list_of_triples, object_qcodes

In [462]:
def get_all_human_objects(list_of_qcodes):
    """
    Filters wikidata objects based on provided Qcodes. Returns only objects which are instance_of human.

    Args:
        list_of_qcodes example: ['Q255', Q'1268']
    
    Returns:
        list of tuples representing humans (qcode, wikidata object url, name)
    """
    #  UWAGA:  Z jakiegoś powodu query poniżej zwraca wielokrotnie zduplikowane wyniki, pewnie błąd leży po stronie twórców biblioteki SPARQLWrapper.
    #          To samo zapytanie wprwadzowne w https://query.wikidata.org/ zwraca normalne, nie powielane wyniki.
    #          Poprawiłem to przy przygotowywaniu zwracanego wyniku
    list_of_wdqcodes = ["wd:" + qcode for qcode in list_of_qcodes]
    query_check_objects = """SELECT ?qcode ?qcodeLabel WHERE {{
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,pl". }}
    VALUES ?qcode {{ {} }}
    ?qcode wdt:P31 wd:Q5.
    }}""".format(' '.join(list_of_wdqcodes))
    endpoint_url="https://query.wikidata.org/sparql"
    humans = get_sparql_results(endpoint_url, query_check_objects)
    urls_qcodes_names = []
    for r in humans["results"]["bindings"]:
        url_qcode_and_name = (r['qcode']['value'][r['qcode']['value'].rfind('Q'):], r['qcode']['value'], r['qcodeLabel']['value'])
        if url_qcode_and_name not in urls_qcodes_names:
            urls_qcodes_names.append(url_qcode_and_name)
    return urls_qcodes_names
    

In [463]:
# rdf_dict = dict()
# ALL_VISITED_QCODES = []
# def recursive_get_rdf_info(input_qcodes, level=3):
#     """For each provided qcode gets all rdf triples from wikidata. Also gets all rdf triples for humans associated with provided qcodes.
#     If level=2, then also gets all rdf triples for humans associated with humans associated provided qcodes, etc...

#     Args:
#         input_qcodes (List): list of qcodes
#         level (int, optional): "Depth of the recursive tree". Defaults to 3.

#     Returns:
#         Dict: Dictionary with qcodsccanocfdsgjwkdbfj:)lk]dsgdsa TODO ogarnąć docstringa i funkcję(przyspieszyć/napisać innym sposobem) 
#     """
#     print("Doing: ", input_qcodes)
#     # rdf_dict = dict()
#     global rdf_dict
#     global ALL_VISITED_QCODES
#     current_humans = []
#     length = len(input_qcodes)
#     for i, Q in enumerate(input_qcodes):
#         if Q in ALL_VISITED_QCODES:  # TODO HERE
#             continue
#         triples, all_qcodes = get_all_triples_and_qcodes(Q, allow_literals=True)
#         rdf_dict[Q] = triples
#         ALL_VISITED_QCODES.append(Q)
#         tuples = get_all_human_objects(all_qcodes)
#         humans = [t[0] for t in tuples]  # human qcodes associated with Q qcode
#         for new_qcode in humans:
#             if new_qcode not in current_humans:
#                 current_humans.append(new_qcode)
#             # triples, all_qcodes = get_all_triples_and_qcodes(new_qcode, allow_literals=True)  # czy to jest tu potrzebne??? moze w kolejnym kroku zrobimy get all triples and qcodes tam wyzej, gdy ten qcode czlowiek bedzie w inpucie funkcji.   ... ALL_VISITED_QCODES
#             # rdf_dict[new_qcode] = triples
#         print(f"Level {level} {i+1}/{length}")
#     if level > 1:
#         print("Done: ", input_qcodes)
#         return recursive_get_rdf_info(current_humans, level=level-1)
#     else:  # level == 1
#         print("Done: ", input_qcodes)
#         return rdf_dict

In [464]:
rdf_dict = dict()
ALL_VISITED_QCODES_WITH_GATHERED_RDF_INFO = []
def recursive_get_rdf_info(input_qcodes, level=3):
    """For each provided qcode gets all rdf triples from wikidata. Also gets all rdf triples for humans associated with provided qcodes.
    If level=2, then also gets all rdf triples for humans associated with humans associated provided qcodes, etc...

    Args:
        input_qcodes (List): list of qcodes
        level (int, optional): "Depth of the recursive tree". Defaults to 3.

    Returns:
        Dict: Dictionary with qcodsccanocfdsgjwkdbfj:)lk]dsgdsa TODO ogarnąć docstringa i funkcję(przyspieszyć/napisać innym sposobem) 
    """
    print("Doing: ", input_qcodes)
    global rdf_dict
    global ALL_VISITED_QCODES_WITH_GATHERED_RDF_INFO
    current_humans = []
    length = len(input_qcodes)

    for i, Q in enumerate(input_qcodes):
        if Q in ALL_VISITED_QCODES:
            continue
        triples, all_qcodes = get_all_triples_and_qcodes(Q, allow_literals=True)
        rdf_dict[Q] = triples
        ALL_VISITED_QCODES_WITH_GATHERED_RDF_INFO.append(Q)
        tuples = get_all_human_objects(all_qcodes)
        humans = [t[0] for t in tuples]  # human qcodes associated with Q qcode

        for new_qcode in humans:
            if new_qcode not in current_humans and new_qcode not in ALL_VISITED_QCODES_WITH_GATHERED_RDF_INFO:
                current_humans.append(new_qcode)

        # for new_qcode in humans:
        #     if new_qcode not in current_humans:
        #         current_humans.append(new_qcode)
        #     triples, all_qcodes = get_all_triples_and_qcodes(new_qcode, allow_literals=True)  # czy to jest tu potrzebne??? moze w kolejnym kroku zrobimy get all triples and qcodes tam wyzej, gdy ten qcode czlowiek bedzie w inpucie funkcji.   ... ALL_VISITED_QCODES
        #     rdf_dict[new_qcode] = triples

        #     if Q in ALL_VISITED_QCODES_WITH_GATHERED_RDF_INFO:
        #         continue

        print(f"Level {level} {i+1}/{length}")
    if level > 0:
        print("Done: ", input_qcodes)
        return recursive_get_rdf_info(current_humans, level=level-1)
    else:  # level == 0
        print("Done: ", input_qcodes)
        return rdf_dict

In [465]:
ALL_LIST = list()
ALL_DICT = dict()
def return_humans(input_humans, stop_level=4):
    global ALL_DICT
    global ALL_LIST
    if stop_level==0:
        print("reached stop level, not doing ", input_humans)
        return ALL_DICT
    print(f"doing {stop_level}", input_humans)
    humans_new_input = []
    for Q in input_humans:
        if Q not in ALL_LIST:
            triples, qcodes = get_all_triples_and_qcodes(Q, allow_literals=True)
            ALL_DICT[Q] = triples
            ALL_LIST.append(Q)
            humans_new_input += [t[0] for t in get_all_human_objects(qcodes)]
    print("ALL_LIST len = ", len(ALL_LIST))
    return return_humans(humans_new_input, stop_level=stop_level-1)
    

# TESTY

In [466]:
# test_list = ['Q4531778', 'Q102218631', 'Q4410600', 'Q4531778', 'Q112605288', 'Q4535388', 'Q4535389', 'Q4535407', 'Q28500400', 'Q1247154', 'Q4535403', 'Q4535407', 'Q19478933', 'Q110160394', 'Q4375007', 'Q4535407', 'Q12142901', 'Q4375007', 'Q4394319', 'Q4535407', 'Q65034322', 'Q101208715', 'Q102432232', 'Q105839650', 'Q115013896', 'Q55103281', 'Q105839650', 'Q4719881', 'Q2026959', 'Q4719881', 'Q61982644', 'Q107294414', 'Q107294416', 'Q4129572', 'Q4719881', 'Q4719881', 'Q112384645', 'Q1964761', 'Q2067058', 'Q2380794', 'Q4125775', 'Q4155513', 'Q29358858', 'Q61198110', 'Q235611', 'Q1964761', 'Q2380794', 'Q4125775', 'Q21644012', 'Q21711184', 'Q61198110', 'Q104706141', 'Q235611', 'Q1964761', 'Q2067058', 'Q4125775', 'Q61198110', 'Q235611', 'Q1964761', 'Q2067058', 'Q2380794', 'Q2713079', 'Q4125760', 'Q21723130', 'Q61198091', 'Q104634182', 'Q235611', 'Q61198122', 'Q3335862', 'Q3784285', 'Q3784470', 'Q19916444', 'Q61198167', 'Q61198191', 'Q109127940', 'Q2834383', 'Q3784470', 'Q4313557', 'Q61198191', 'Q2834383', 'Q3784285', 'Q3784470', 'Q109127940', 'Q109796556', 'Q109796591', 'Q1651756', 'Q3911020', 'Q3960168', 'Q4111755', 'Q5362962', 'Q5540511', 'Q5623705', 'Q5801949', 'Q7345451', 'Q7509123', 'Q75253381', 'Q75268252', 'Q75276782', 'Q75278573', 'Q76281589', 'Q76297706', 'Q76297708', 'Q337595', 'Q1063510', 'Q3911020', 'Q4111755', 'Q4719715', 'Q7345451', 'Q75253381', 'Q75268252', 'Q75274768', 'Q75276782', 'Q75278573', 'Q75281989', 'Q75631188', 'Q337595', 'Q1651756', 'Q3723423', 'Q3911020', 'Q4111579', 'Q7345451', 'Q75251358', 'Q75253381', 'Q75268252', 'Q75276782', 'Q75278573', 'Q75280057', 'Q75347169', 'Q75347173', 'Q337595', 'Q1651756', 'Q3911020', 'Q4111755', 'Q7794690', 'Q23614621', 'Q75249398', 'Q75251362', 'Q75260050', 'Q75268252', 'Q75276782', 'Q75278573', 'Q75677852', 'Q337595', 'Q1651756', 'Q3911020', 'Q4111755', 'Q5537356', 'Q75253381', 'Q75276782', 'Q75278573', 'Q337595', 'Q1651756', 'Q3911020', 'Q4111755', 'Q7327762', 'Q7327763', 'Q7345437', 'Q75253381', 'Q75268252', 'Q75278573', 'Q75281883', 'Q75281885', 'Q75940386', 'Q337595', 'Q1651756', 'Q3911020', 'Q4111755', 'Q5935407', 'Q75253381', 'Q75268252', 'Q75276782', 'Q4527260', 'Q15060960', 'Q15407959', 'Q18280384', 'Q84720837', 'Q88944302', 'Q984563', 'Q1506392', 'Q11307766', 'Q15060960', 'Q18280384', 'Q102177202', 'Q104368599', 'Q778295', 'Q4527275', 'Q15064867', 'Q20165319', 'Q23925203', 'Q75318089', 'Q75318092', 'Q75318111', 'Q75318160', 'Q101242405', 'Q4125786', 'Q4125787', 'Q4346487', 'Q16389857', 'Q16478232', 'Q16798198', 'Q33381132', 'Q61638016', 'Q76352808', 'Q116247736', 'Q1439976', 'Q4125791', 'Q16389857', 'Q16478232', 'Q16798198', 'Q75391796', 'Q1439976', 'Q4125787', 'Q4527275', 'Q16389857', 'Q16798198', 'Q103942821', 'Q1439976', 'Q4125787', 'Q4523074', 'Q16389857', 'Q16478232', 'Q61637775', 'Q75246898', 'Q75970451', 'Q75970453', 'Q75970454', 'Q75970455', 'Q1439976', 'Q16389857', 'Q76287573', 'Q76352811', 'Q101081838', 'Q1439976', 'Q4308386', 'Q16389857', 'Q4076493', 'Q105957578', 'Q106547622', 'Q106547728', 'Q4076491', 'Q105957578', 'Q106547862', 'Q367109', 'Q4374909', 'Q4767540', 'Q11731188', 'Q91813941', 'Q91813952', 'Q367109', 'Q4374916', 'Q5975640', 'Q9357360', 'Q11125465', 'Q11731188', 'Q11766641', 'Q11769839', 'Q36742', 'Q367109', 'Q1361286', 'Q9256429', 'Q367109', 'Q4374916', 'Q4767540', 'Q5703795', 'Q7599212', 'Q367109', 'Q4374911', 'Q4723654', 'Q6320794', 'Q6761317', 'Q7362134', 'Q12110477', 'Q16200028', 'Q220030', 'Q4723654', 'Q5298213', 'Q6320538', 'Q6420482', 'Q7702125', 'Q16200028', 'Q85538', 'Q220030', 'Q898557', 'Q2506976', 'Q4723654', 'Q6320794', 'Q60391559', 'Q1361286', 'Q2439671', 'Q2832518', 'Q3672163', 'Q5368719', 'Q6306549', 'Q11742831', 'Q975395', 'Q3672163', 'Q5368719', 'Q6306549', 'Q6320568', 'Q11742831', 'Q349948', 'Q690123', 'Q975395', 'Q2439671', 'Q5368719', 'Q6306549', 'Q6761676', 'Q11742831', 'Q36742', 'Q975395', 'Q2439671', 'Q3672163', 'Q6306549', 'Q11742831', 'Q550652', 'Q558139', 'Q975395', 'Q2439671', 'Q3672163', 'Q5368719', 'Q6306549', 'Q100151843', 'Q9151537', 'Q11750339', 'Q975395', 'Q1361286', 'Q4374905', 'Q6968070', 'Q9162035', 'Q11771789', 'Q41414', 'Q2832518', 'Q6968070', 'Q7599129', 'Q9162035', 'Q11742832', 'Q11771789', 'Q2832518', 'Q2972188', 'Q4374905', 'Q6761317', 'Q9162035', 'Q11771789', 'Q2832518', 'Q4374905', 'Q6968070', 'Q9162035', 'Q11686963', 'Q11771794', 'Q25776', 'Q63184609', 'Q25776', 'Q47145827', 'Q65922309', 'Q65922485', 'Q105697795', 'Q105697796', 'Q25776', 'Q11749206', 'Q17552544', 'Q27556004', 'Q105697795', 'Q105697796', 'Q9262636', 'Q107380248', 'Q9262636', 'Q11814387', 'Q3647036', 'Q4767640', 'Q9155230', 'Q9356981', 'Q11731319', 'Q26611', 'Q3920109', 'Q6373177', 'Q11731319', 'Q3136526', 'Q3917274', 'Q11737678', 'Q17045211', 'Q22911301', 'Q63184609', 'Q63184639', 'Q2991218', 'Q3136526', 'Q3917274', 'Q11737678', 'Q17063837', 'Q63184609', 'Q64031190', 'Q380545', 'Q11735941', 'Q11735941', 'Q11735941', 'Q27118276', 'Q27539092', 'Q102536331', 'Q646034', 'Q4394264', 'Q9341597', 'Q63184593', 'Q102536340', 'Q111451587', 'Q4374912', 'Q6836005', 'Q63184593', 'Q105085428']
# t, q = get_all_triples_and_qcodes(Qcode=['Q254', 'Q255', 'Q1268', 'Q9148844', 'Q65095948', 'Q16588072']+test_list, allow_literals=True)
# t, q = get_all_triples_and_qcodes(Qcode='Q254', allow_literals=True)  # TODO sprawdzić działanie get_all_triples_and_qcodes z listą qcodów jako inputem, może będzie szybsze niż pojedyncze szukanie rdfów

# Input: List of persons ```(names from .txt file)```

In [467]:
with open("unique_persons.txt", 'r') as f:
    input = [name for name in f.read().splitlines() if len(name.split())!=1]
# input = ['Beethoven', 'Rachmaninoff', 'Mahler', 'Delta rzeki Okawango', 'Susanne Gibson-Milne', 'asfa fafasfs', 'sekwoja', 'Pierre']
input = ['Alojzy Janowicz (pułkownik)', 'Dariusz Gotlib', 'Piotr Franciszek Branicki']
# input = ['Waldemar Izdebski', 'Dariusz Gotlib', 'Jacek Marciniak', 'Robert Olszewski', 'Michał Wyszomirski']

In [468]:
print(len(input))
input

3


['Alojzy Janowicz (pułkownik)', 'Dariusz Gotlib', 'Piotr Franciszek Branicki']

# Q-codes for names from input_list ```(Q-codes from wikidata search page)```

In [469]:
original_persons = scrape_qcodes(list=input, humans_only=True, first_only=True)  # 8m 30s dla 399 osób z listów z pliu .txt

Scraping... 1/4 ... Alojzy Janowicz (pułkownik)
Scraping... 2/4 ... Dariusz Gotlib
Scraping... 3/4 ... Piotr Franciszek Branicki
Scraping... 4/4 ... Alojzy Janowicz 


In [470]:
original_persons.sort(key=lambda d: d['search_name'])
print(len(original_persons))
original_persons

3


[{'search_name': 'Alojzy Janowicz ',
  'page_name': 'Alojzy Janowicz',
  'description': None,
  'Qcode': 'Q9148844',
  'wiki_url': 'https://www.wikidata.org/wiki/Q9148844'},
 {'search_name': 'Dariusz Gotlib',
  'page_name': 'Dariusz Gotlib',
  'description': 'Polish engineer',
  'Qcode': 'Q65095948',
  'wiki_url': 'https://www.wikidata.org/wiki/Q65095948'},
 {'search_name': 'Piotr Franciszek Branicki',
  'page_name': 'Q16588072',
  'description': None,
  'Qcode': 'Q16588072',
  'wiki_url': 'https://www.wikidata.org/wiki/Q16588072'}]

In [471]:
input_qcodes = [d['Qcode'] for d in original_persons]
input_qcodes

['Q9148844', 'Q65095948', 'Q16588072']

In [472]:
input_persons_wo_qcode = set(input) - {d['search_name'] for d in original_persons}
# with open ("input_persons_wo_qcode.txt", "w") as f:
#     f.write(str(input_persons_wo_qcode))
print(len(input_persons_wo_qcode))
input_persons_wo_qcode

1


{'Alojzy Janowicz (pułkownik)'}

In [473]:
extra_persons_w_qcode = {d['search_name'] for d in original_persons} - set(input)
# with open ("extra_persons_w_qcode.txt", "w") as f:
#     f.write(str(extra_persons_w_qcode))
print(len(extra_persons_w_qcode))
extra_persons_w_qcode

1


{'Alojzy Janowicz '}

# Humans associated with names from original_list ```(N levels of depth...)```

In [474]:
# SLOW... prszyspeszyć działanie funkcji TODO
# oho, już widzę jeden błąd... dla level=5 i input = ['Waldemar Izdebski', 'Dariusz Gotlib', 'Jacek Marciniak', 'Robert Olszewski', 'Michał Wyszomirski']... (naprawioned? dodałem ALL_VISITED_QCODES... work in progress)

# rdf_info = recursive_get_rdf_info(input_qcodes, level=5)
rdf_info = return_humans(input_qcodes, stop_level=6)

doing 6 ['Q9148844', 'Q65095948', 'Q16588072']
ALL_LIST len =  3
doing 5 ['Q1349049', 'Q9253321', 'Q26882450', 'Q105679898']
ALL_LIST len =  7
doing 4 ['Q2028738', 'Q2670161', 'Q4125793', 'Q8073362', 'Q11737046', 'Q16588072', 'Q26882450', 'Q26882457', 'Q2628182', 'Q16588072', 'Q40989542', 'Q1349049', 'Q16588072', 'Q24263182', 'Q16588072']
ALL_LIST len =  16
doing 3 ['Q1349049', 'Q2670161', 'Q4095822', 'Q4125793', 'Q6375196', 'Q7386598', 'Q8073362', 'Q9252846', 'Q9382095', 'Q11737046', 'Q11742884', 'Q11750335', 'Q26882457', 'Q97497657', 'Q1349049', 'Q2028738', 'Q4125793', 'Q8073362', 'Q11737046', 'Q26882457', 'Q61809857', 'Q61809861', 'Q466151', 'Q1349049', 'Q2028738', 'Q2670161', 'Q4125779', 'Q8073362', 'Q11737046', 'Q26882457', 'Q61637815', 'Q1349049', 'Q2028738', 'Q2670161', 'Q4125793', 'Q4801655', 'Q9139711', 'Q11737046', 'Q26882457', 'Q1349049', 'Q2028738', 'Q2670161', 'Q4125793', 'Q6961400', 'Q8073362', 'Q9146738', 'Q26882457', 'Q1349049', 'Q2028738', 'Q2670161', 'Q4125793', 'Q807

In [475]:
# ^ czas działania recursive_get_rdf_info:
# rdf_info = recursive_get_rdf_info(input_qcodes, level=2)      # działało 3m 36s dla input-codes=['Q131861', 'Q255'] i zwróciło rdf_info dla 228 osób (104772 linijek słownika)
# rdf_info = recursive_get_rdf_info(input_qcodes, level=2)      # działało 41s dla input-codes=['Q131861'] i zwróciło rdf_info dla 36 osób (17376 linijek słownika)

# rdf_info = recursive_get_rdf_info(input_qcodes, level=3)      # działało 423m 35s dla input_codes z pliku .txt (399 osób (>1 słowo) z listów Chopina i Paderewskiego); len(rdf_info.keys())=13171; rdf_info 5 mln linijek;
                                                                # 180 MB json z indent=4; 117MB json w jednej linijce;

# recursive_get_rdf_info(input_qcodes, level=3) 18m 40s ['Q9148844', 'Q65095948', 'Q16588072', 'Q254']  len(rdf_info.keys())=571
# recursive_get_rdf_info(input_qcodes, level=3) 1m 7s ['Q9148844', 'Q65095948', 'Q16588072']  len(rdf_info.keys())=33

In [476]:
# input = ['Alojzy Janowicz (pułkownik)', 'Dariusz Gotlib', 'Piotr Franciszek Branicki']
# 4m 40s len(rdf_info.keys())=236 len(all_humans) = 643 recursive_get_rdf_info(input_qcodes, level=5) V poprzednia z jednym ALL_COSTAM
# 1m 2s  len(rdf_info.keys())=88 len(all_humans) = 236 recursive_get_rdf_info(input_qcodes, level=5) V terazniejsza - zakomentowana
# 2m 54s len(rdf_info.keys())=236 len(all_humans) = 643 recursive_get_rdf_info(input_qcodes, level=6) V terazniejsza - zakomentowana
# 3m 50s len(rdf_info.keys())=236 len(all_humans) = 643 recursive_get_rdf_info(input_qcodes, level=5) V terazniejsza eskperymenty
# 2m 42s len(rdf_info.keys())=236 len(all_humans) = 643 return_humans(input_qcodes, stop_level=6) V terazniejsza return_humans
print(f"{len(rdf_info.keys())=}")
rdf_info

len(rdf_info.keys())=236


{'Q9148844': [('http://www.wikidata.org/entity/Q9148844',
   'http://www.wikidata.org/entity/P20',
   'http://www.wikidata.org/entity/Q171689',
   'Alojzy Janowicz',
   'place of death',
   '12th arrondissement of Paris'),
  ('http://www.wikidata.org/entity/Q9148844',
   'http://www.wikidata.org/entity/P21',
   'http://www.wikidata.org/entity/Q6581097',
   'Alojzy Janowicz',
   'sex or gender',
   'male'),
  ('http://www.wikidata.org/entity/Q9148844',
   'http://www.wikidata.org/entity/P27',
   'http://www.wikidata.org/entity/Q172107',
   'Alojzy Janowicz',
   'country of citizenship',
   'Polish–Lithuanian Commonwealth'),
  ('http://www.wikidata.org/entity/Q9148844',
   'http://www.wikidata.org/entity/P31',
   'http://www.wikidata.org/entity/Q5',
   'Alojzy Janowicz',
   'instance of',
   'human'),
  ('http://www.wikidata.org/entity/Q9148844',
   'http://www.wikidata.org/entity/P53',
   'http://www.wikidata.org/entity/Q63531372',
   'Alojzy Janowicz',
   'family',
   'Janowiczowie her

In [477]:
# import json
# with open("rdf_info.json", "w") as f:
#     json.dump(rdf_info, f)  # no idntent arg if you want smaller filesize

In [478]:
qcodes_to_check = set()
for Q in rdf_info.keys():
    qcodes_to_check.add(Q)
    for rdf_tuple in rdf_info[Q]:
        if rdf_tuple[4] == "FactGrid item ID":
            continue
        object_url = rdf_tuple[2]
        qcode = object_url[object_url.rfind('Q'):]
        if len(qcode) > 1:
            if qcode[1:].isdigit():
                qcodes_to_check.add(qcode)
                if (qcode=='Q383232'):
                    print(Q, rdf_info[Q])
print(f"{len(qcodes_to_check) = }")

qcodes_to_check_split = []
n_size = 500  # splitting qcodes_to_check into lists of n_size (500) elements, because URI cannot be too large in function get_all_human_objects()
length = len(qcodes_to_check)
temp = []
for i, qcode in enumerate(qcodes_to_check):
    temp.append(qcode)
    if i%n_size == n_size-1:
        qcodes_to_check_split.append(temp)
        temp = []
    if i == length-1:
        if len(temp) > 0:
            qcodes_to_check_split.append(temp)
print(f"{len(qcodes_to_check_split) = }")

all_humans = []
n_lists = len(qcodes_to_check_split)
for i, lista in enumerate(qcodes_to_check_split):
    temp_list = get_all_human_objects(lista)
    all_humans += temp_list
# ^ 1m 30s dla len(qcodes_to_check) = 60829

len(qcodes_to_check) = 1791
len(qcodes_to_check_split) = 4


In [479]:
print(f"{len(all_humans) = }")
all_humans

len(all_humans) = 643


[('Q9312', 'http://www.wikidata.org/entity/Q9312', 'Immanuel Kant'),
 ('Q36450', 'http://www.wikidata.org/entity/Q36450', 'Catherine II of Russia'),
 ('Q43939', 'http://www.wikidata.org/entity/Q43939', 'Anselm of Canterbury'),
 ('Q45662', 'http://www.wikidata.org/entity/Q45662', 'Klemens von Metternich'),
 ('Q54019',
  'http://www.wikidata.org/entity/Q54019',
  'Stanisław August Poniatowski'),
 ('Q60070',
  'http://www.wikidata.org/entity/Q60070',
  'Friedrich Wilhelm Joseph Schelling'),
 ('Q79025', 'http://www.wikidata.org/entity/Q79025', 'Walter Scott'),
 ('Q134189', 'http://www.wikidata.org/entity/Q134189', 'Plotinus'),
 ('Q154751', 'http://www.wikidata.org/entity/Q154751', 'Nicholas of Cusa'),
 ('Q168004',
  'http://www.wikidata.org/entity/Q168004',
  'Friedrich Heinrich Jacobi'),
 ('Q176163', 'http://www.wikidata.org/entity/Q176163', 'August Cieszkowski'),
 ('Q272161', 'http://www.wikidata.org/entity/Q272161', 'Anne Isabella Byron'),
 ('Q454204', 'http://www.wikidata.org/entity/Q4

# Creating neo4j graph

## connect to database

In [480]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
driver.verify_connectivity()
session = driver.session(database="wikidatadev")

## create Person nodes

In [481]:
all_humans_qcodes = []
for human in all_humans:
    qcode, wd_url, name = human
    all_humans_qcodes.append(qcode)
    try:
        cypher_query = f"""MERGE (:Person{{name: "{name}", wd_url: "{wd_url}"}})"""
        session.run(cypher_query)
    except CypherSyntaxError:  #  Karol Stanisław "Panie Kochanku" Radziwiłł (Q386484) 
        cypher_query = f"""MERGE (:Person{{name: '{name}', wd_url: '{wd_url}'}})"""
        session.run(cypher_query)
# ^ 2m16s dla len(all_humans) =~ 30_000 (powstałe z 399 osób z inputu)

In [482]:
cypher_query = f"""
CREATE INDEX person_wd_idx IF NOT EXISTS
FOR (n:Person)
ON (n.wd_url)
"""
session.run(cypher_query)

cypher_query = f"""
CREATE INDEX person_name_idx IF NOT EXISTS
FOR (n:Person)
ON (n.name)
"""
session.run(cypher_query)

# jak sprawdzić, czy indexy się założyły?

## create other nodes, edges & connect to Person nodes

In [483]:
# # SLOW
# ERRORS = []
# length = len(rdf_info.keys())
# print("Creating edges & object nodes for: ")
# for i, qcode in enumerate(rdf_info.keys()):
#     print(f"{qcode} {i+1} / {length}")
#     for t in rdf_info[qcode]:
#         object_name, object_url = t[5].replace("'", "\\'"), t[2].replace("'", "\\'")

#         object_label = "wdLabel"
#         object_qcode = object_url[object_url.rfind("Q"):]
#         if object_qcode in all_humans_qcodes:
#             object_label = "Person"
#         else:
#             cypher_query = f"""MERGE (:{object_label}{{name: '{object_name}', wd_url: '{object_url}'}})"""
#             try:
#                 session.run(cypher_query)
#             except Exception as e:
#                 ERRORS.append(("MERGE NODE", object_url, cypher_query, e))

#         cypher_illegal_edgename_chars = ["'", ",", "(", ")", "-", ".", "/", "&", "’", "+", "+d'-", ":"]
#         edge_name = t[4].replace(' ', '_').replace("–", "_")
#         for c in cypher_illegal_edgename_chars:
#             edge_name = edge_name.replace(c, "")
#         edge_name = edge_name.upper()
#         edge_url = t[1]

#         subject_url = t[0].replace("'", "\\'")
#         person_name = t[3].replace("'", "\\'")
#         try:
#             s = f"""
#             MATCH (p:Person), (l:{object_label}) 
#             WHERE p.wd_url="{subject_url}" AND l.wd_url='{object_url}'
#             MERGE (p)-[:`{edge_name}` {{wd_url: '{edge_url}', source: 'wikidata'}}]->(l)
#             """
#             session.run(s)
#         except Exception as e:
#             ERRORS.append(("MERGE EDGE", object_url, s, e))

In [484]:
# ^ 9s dla len(rdf_info.keys()) = 35 len(all_humans) = 88
# ^ 45s dla len(rdf_info.keys()) = 36 len(all_humans) = 170
# ^ 936m 52s dla len(rdf_info.keys()) = 13171 len(all_humans) = 29966

In [485]:
# ERRORS
# ^ lista ERRORS była pusta dla 936m 52s dla len(rdf_info.keys()) = 13171 len(all_humans) = 29966. To pewnie jakiś błąd jupytera(?), może za długo komórka działała. Jakeiś errory musiały się pojawić....

In [486]:
# WORK IN PROGRESS 
edges_csv_fp = r"C:\GraphProject\Places_People_and_Events\Krzysztof\edges.csv"
wdLabel_nodes_csv_fp =  r"C:\GraphProject\Places_People_and_Events\Krzysztof\wdLabel_nodes.csv"

non_person_node_label = "wdLabel"
unique_edge_names = set()
with open(edges_csv_fp, "w", newline="") as f:
    with open(wdLabel_nodes_csv_fp, "w", newline="") as f2:
        writer = csv.writer(f, delimiter=";")
        fields = ["subject_url", "edge_url", "object_url", "subject_name", "edge_name", "object_name", "object_label"]
        writer.writerow(fields)

        writer2 = csv.writer(f2, delimiter=";")
        fields2 = ["object_name", "object_url"]
        writer2.writerow(fields2)

        length = len(rdf_info.keys())
        for i, qcode in enumerate(rdf_info.keys()):
            for t in rdf_info[qcode]:
                subject_url = t[0].replace("'", "\\'")
                subject_name = t[3].replace("'", "\\'")
                edge_url = t[1]
                edge_name = t[4].replace(' ', '_').replace("–", "_")
                cypher_illegal_edgename_chars = ["'", ",", "(", ")", "-", ".", "/", "&", "’", "+", "+d'-", ":", ";"]
                for c in cypher_illegal_edgename_chars:
                    edge_name = edge_name.replace(c, "")
                edge_name = edge_name.upper()
                unique_edge_names.add(edge_name)
                object_url = t[2].replace("'", "\\'")
                object_name = t[5].replace("'", "\\'")
                object_qcode = object_url[object_url.rfind("Q"):]
                if object_qcode in all_humans_qcodes:
                    object_label = "Person"
                else:
                    object_label = non_person_node_label

                    row2 = [object_name, object_url]
                    writer2.writerow(row2)
                
                row = [subject_url, edge_url, object_url, subject_name, edge_name, object_name, object_label]
                writer.writerow(row)

In [487]:
# https://neo4j.com/docs/cypher-manual/current/clauses/call-subquery/#subquery-call-in-transactions
# https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/
# UWAGA: nie można przypisywać labelek z pliku csv jako np. line.label
# EHH: https://community.neo4j.com/t/create-edges-and-their-labels-from-load-csv/45325
# https://stackoverflow.com/questions/47938032/how-to-get-relation-from-csv-in-neo4j-cypher-using-csv-load
# possible rozwiązanie: zrobić milion plików .csv, każdy odpowiadający innej nazwie krawędzi... ale najpierw przetestować to komórkę niżej 

query = f"""
LOAD CSV WITH HEADERS FROM 'file:///{wdLabel_nodes_csv_fp}' AS line FIELDTERMINATOR ';'
CALL {{
  WITH line
  MERGE (:wdLabel {{ name: line.object_name, wd_url: line.object_url }})
}} IN TRANSACTIONS
"""
session.run(query)

In [488]:
# o_O coś w końcu działa... przetestować szybkość wykonywania TODO
for edge_name in unique_edge_names:
  query2 = f"""
  LOAD CSV WITH HEADERS FROM 'file:///{edges_csv_fp}' AS line FIELDTERMINATOR ';'
  CALL {{
    WITH line 
    MATCH (p:Person {{wd_url: line.subject_url}}), (o{{wd_url: line.object_url}})  
    WHERE line.edge_name = "{edge_name}"
    MERGE (p)-[:{edge_name} {{wd_url: line.edge_url, source: 'wikidata'}}]->(o)
  }} IN TRANSACTIONS
  """
  # print(query2) # HERE o:wdLabel
  session.run(query2)

## delete all nodes

In [493]:
# # DELETE ALL
# s="""MATCH (n)
# CALL { WITH n
# DETACH DELETE n
# } IN TRANSACTIONS;"""
# session.run(s)

## session close

In [490]:
# session.close()

# Komentarze
Zbiór niezbyt dobrze uporządkowanych myśli i nieścisłych wyrażeń *(informacje dla mnie, po jakimś czasie i tak przestałem aktualizować tę sekcję)*

Co zrobiłem:
* uporządkowałem kod
* odrzuciłem pojedyncze nazwy z listy osób, np. "Pierre"
* ```get_all_triples_and_qcodes()``` dodałem opcję zostawienia tych objects, które nie są Qcodami w wikidata (różne literały, często są to jakieś id w muzeach czy innych zbiorach danych, wcześniej to odrzuciłem bo wydało mi się nieprzydatne) 
* ^ ale wd_url będzie wtedy literałem, a nie urlem w grafie. Nie jest to chyba duży problem i może tak zostać.
* dla krawędzi: uppercase nazw i dodanie atrybutu ```source: 'wikidata' ```
* dla każdego subject z listy sprawdzenie wszystkich powiązanych objects i zapisanie qcodów tych obiektów, które są ```instance_of``` --> ```human```
* ```scrape_qcodes(list, humans_only=False, first_only=False)``` dodałem sposób na dostanie Qcode tylko pierwszego znalezionego człowieka na wikidata
* zrezygnowałem z atrybutu ```description``` dla scrapeowanych qcodów (w grafie)
* rekurencyjne pobieranie info o ludziach połączonych z ludźmi z inputu

Kilka problematycznych kwestii:
* a może nie brać osób z listów Paderewskiego, a zostawić tylko osoby z listów Chopina?
* osoba, o którą nam chodzi nie jest pierwszym wynikiem na liście wyszukiwania w wikidata
* czy kontrolujemy manualnie poprawność osób dodanych z wikidata? (chodzi o osoby z input_list)
* biblioteka SPARQLWrapper nie działa dobrze, zwraca często zmultiplikowane wyniki i spowalnia działanie programu
* osoby, które są na najniższym poziome (liście drzewa) nie mają żadnych atrybutów (trójek rdf), bo kiedyś ten łańcuch trzeba skończyć. Pytanie w jaki sposób. 
* w grafie prawdopodobnie będą błędy, np. randomowe osoby nazywające się tak samo jak np. te wspomniane w listach, a tych wspomnianych nie będzie w grafie
* będą sobie pływały randomowe węzły osób w tym grafie, np. Q383232 (Afiat Yuris Wirawan, gracz badmintona) bo np Rachmaninov ma "FactGrid item ID": Q383232
* ^ EDIT:   naprawiłem to dla tego specyficznego przypadku krawędzi "FactGrid item ID", u nich wszystkie ID są postaci takiej jak Qcody na wikidata
* ^         ale mogą się pojawić inne randomowe osoby i przez to nie będzie tworzony węzęł z obiektem o nazwie np. Q123 i nie będzie krawędzi między subjectem(osobą) a obiektem/randomową osobą

['Etienne',
 'Perrichet',
 'Pierre',
 'Alexis',
 'Grandville',
 'Rafael',
 'Voltaire']

Granville https://www.wikidata.org/wiki/Q744442 ???

Voltaire https://www.wikidata.org/wiki/Q9068 ???

Pierre służący Fryderyka Chopina ???


--- na razie robię sobie przerwę od tego projektu, muszę zacząć robić pracę inżynierską i mam też inny projekt

-- https://query.wikidata.org/ do rozważenia jakby ktoś chciał się zająć pobieraniem innych danych (nie dotyczących osób)

# TODO



* Ctrl + f "TODO" w kodzie - things to work on first
* Zbyt długo trwa robienie grafu. Może zrobienie csv i potem wczytywanie csv do neo4j będzie szybsze niż pojedyncze robienie węzłów i krawędzi. Zrobić indeksy, zwiększyć limity pamięci w neo4j desktop, zrobić CSV do krawędzi i LOAD CSV, CALL IN TRANSACTIONS. Do zrobienia.
* Zbyt długo trwa recursive_get_rdf_info. Napisać szybszą funkcję, dającą ten sam wynik. 
* Może by tak oznaczać osoby (węzeł :Person), które nie mają atrybutów z wikidata (czyli w grafie powiązań z węzłami wdLabel) jakąś inną etykietą, lub jakimś atrybutem, np. "leaf"/"no_further_info"/"no_". Są to "osoby-liście", na których kończy się rekurencyjne pobieranie info o ludziach. ({all_humans} - {rdf_info.keys()}). A może nie trzeba i zostawić tak jak jest?
* Biblioteka SPARQLWrapper nie działa dobrze, zwraca często zmultiplikowane wyniki i spowalnia działanie programu, można spróbować użyć innej biblioteki lub samemu napisać.
* Może nie brać osób z listów Paderewskiego, a zostawić tylko osoby z listów Chopina?
* [EDIT] [nieważne, neo4j browser nie wyświetla krawędzi czasami] czemu wielu kompozytorów nie ma OCCUPATION -> composer? np. Robert Schumann... Wygląda na to, że czasamiu niektóre krawędzie się nie tworzą, zbadać przyczynę i naprawić.
* [DONE] Niektórych osób z input listy nie wyszukuje w wikdata. Jak się usunie dopiski w nawiasach, to już tak, np. dla "Józef Michał Poniatowski (książę)".
* [DONE] w np. DATE_OF_BIRTH są jakieś śmieci czasami, np. http://www.wikidata.org/.well-known/genid/c9a8b2f22783c60e6bf383200d9285f7. Pewnie można by w "create other nodes, edges & connect to Person nodes" sprawdzać wartość dla qcodów dat np date_of_birth, date_of_costam.


** TYLKO POMYSŁ: sporo czasu by trzeba poświęcić na sprawdzanie czy pierszy scrape'owany human z wikidata search to osoba której chcemy (czyli w jakiś sposób powiązana z Chopinem/osobami na grafie).


## sample cypher queries

In [491]:
# UWAGA: nie można traktować uzyskanych wyników jako prawdę odzwierdciedlającą rzeczywistość; dane są niekompletne i pewnie niektóre nawet są nieprawdziwe.

# MATCH (n) WHERE n.name =~ ".*Chopin.*" RETURN n

# match (n:Person)-[e:OCCUPATION]-(s) WHERE s.name CONTAINS "composer" return n,s

# MATCH (p:Person)-[l:residence]-(s)
# WHERE p.name=~'.*Chopin' or p.name=~'.*Liszt'
# RETURN p, s

# MATCH (p:Person)-[l:RESIDENCE]-(s)
# WHERE p.name=~'.*Chopin' or p.name=~'.*Beethoven.*'
# RETURN p, s

# MATCH (p:Person)-[l:STUDENT_OF]-(s)
# WHERE p.name=~'.*Frédéric Chopin.*' or p.name=~'.*Franz Liszt.*'
# RETURN p, s

# Match(p:Person)-[e:date_of_birth]-(l:Label) return p, l

# match (n:Person) WHERE n.name CONTAINS "Bee" return n

# MATCH (p:Person)-[e:OCCUPATION]-(s:wdLabel) WHERE s.name =~ ".*singer.*" RETURN p,s

# MATCH w=(p:Person)-[e:INSTRUMENT]-(s:wdLabel)
# WITH s, COLLECT(w) as test
# ORDER BY SIZE(test) DESC
# RETURN s.name, SIZE(test)

# MATCH (p:Person)-[e:DATE_OF_BIRTH]-(d:wdLabel)
# WHERE datetime({year: 1770, month:7}) <= datetime(d.name) < datetime({year: 1771})
# WITH p, d
# ORDER BY d.name DESC
# RETURN p, d, p.name, d.name

# MATCH (p:Person)-[e:HANDEDNESS]-(s:wdLabel), (p)-[i:INSTRUMENT]-(s2:wdLabel)
# WHERE s.name =~ '.*left.*' 
# RETURN *  

# MATCH (p:Person)-[e:HANDEDNESS]-(s:wdLabel)
# WHERE s.name =~ '.*left.*' 
# RETURN *  

# MATCH (p:Person)-[e:DIFFERENT_FROM]-(o)
# WHERE p.name =~ '.*.*'
# RETURN p, o limit 200
